In [96]:
# Esta celda define funciones y librerias necesarias. Ejecutarla siempre al principio
import pandas as pd
import requests
import numpy as np
import time

try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup
    
# Funcion para obtener el HTML de un link
def get_html(url):
    r = requests.get(url)
    html = BeautifulSoup(r.text)
    return html.body

def get_html_safe(url):
    html  = None
    while(True):
        retries = 0
        while(retries < 5):
            try:  
                html = get_html(row['product_url'])
                break
            except:
                retries = retries+1
                print("Fallo la conexion intentando nuevamente en 30 segundos ({}/5)".format(retries))
                time.sleep(30)
                
                
        if retries < 5:
            break
        else:
            input("Maxima cantidad de intentos alcanzada. Verifique conexion y presione cualquier tecla")
            
    
    return html
    

def find_error_in_html(html, key, value):
    if key in ['Ended', 'Listing Ended']:
        return html.find(id=value)
    elif key in ['Sold Out', 'Found Similar', 'Error']:
        return html.find(class_=value)
    else : 
        return None
    
    
def check_good_link(html, product_title):
    match = html.find(id='LeftSummaryPanel')
    if match is not None:
        # Busco el titulo
        match = match.find(id="itemTitle")
        
    # Si encontre el id busco el titulo del producto
    if match is not None:
        if match.a is not None:
            match = (match.a['data-mtdes'] == product_title)
        else:
            match = match.find(text=product_title)
        
    return (match is not None)
        



In [97]:
# Variables necesarias. Estas se pueden cambiar para usar el script como se prefiera
filename = "jordan_review_for_e_rav.xlsx" #Nombre del archivo de entrada
output_filename = "jordan_review_for_e_rav_processed.xlsx" # Nombre del archivo de salida

primer_fila = 0 # Primera fila a procesar. Para comenzar del principio poner 0
ultima_fila = 500 # Ultima fila a procesar. Para procesar todas poner en -1

# Ids a buscar, vienen del HTML
search_product_url = { 'Ended': 'w1-5-_msg',
                       #'Listing Ended': 'w1-6-_msg',
                       'Sold Out': 'outofstock',
                       'Found Similar': 'app-cvip-replacement-message',
                       'Error': 'error-header__headline',
                     }

# Leo el archivo en el dataframe
df = pd.read_excel("../spreadsheets/" + filename)

if ultima_fila > -1:
    df = df[primer_fila:ultima_fila]
elif primera_fila > 0:
    df = df[primera_fila:]

# Saco las columnas que no voy a usar
cols_to_drop = ['site_id', 'MPN (ignore)']
df = df.drop(cols_to_drop, axis="columns")

#Agrego columnas nuevas para los datos procesados
df["YES/NO Procesado"] = np.nan
for key in search_product_url.keys():
    df[key] = np.nan

# Agrego el https:// a todos los links
df['product_line_url'] = df['product_line_url'].apply(lambda s: 'https://'+s)
df['product_url'] = df['product_url'].apply(lambda s: 'https://'+s)
df

,Product_line_id,Product_line_description,product_id,product_title (by seller),product_line_url,product_url,YES/NO,YES/NO Procesado,Found Similar,Sold Out,Ended,Error
0,7117656342,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",264737435029,NIKE AIR JORDAN 1 RETRO + NEUTRAL GREY 2001 SI...,https://www.ebay.com/b/bn_7117656342,https://www.ebay.com/itm/264737435029,y,NaN,NaN,NaN,NaN,NaN
1,7117717160,"[Brand = Jordan, Model = Jordan 'Why Not?' ZER...",164677534195,NEW Nike Jordan Why Not? Zer0.4 FACETASM Unise...,https://www.ebay.com/b/bn_7117717160,https://www.ebay.com/itm/164677534195,y,NaN,NaN,NaN,NaN,NaN
2,7117656574,"[Brand = Jordan, Model = Air Jordan 13, Silhou...",293808623286,Nike Air Jordan 2010 Retro 13 Varsity White Re...,https://www.ebay.com/b/bn_7117656574,https://www.ebay.com/itm/293808623286,y,NaN,NaN,NaN,NaN,NaN
3,7117651224,"[Brand = Jordan, Model = Air Jordan 5, Silhoue...",114780501812,2021 Nike Air Jordan Retro 5 Low Chinese New Y...,https://www.ebay.com/b/bn_7117651224,https://www.ebay.com/itm/114780501812,y,NaN,NaN,NaN,NaN,NaN
4,7117643400,"[Brand = Jordan, Model = Air Jordan 10, Silhou...",334065851300,Size 10 - Jordan 10 Retro NYC 2016,https://www.ebay.com/b/bn_7117643400,https://www.ebay.com/itm/334065851300,y,NaN,NaN,NaN,NaN,NaN
5,7117647055,"[Brand = Jordan, Model = Air Jordan 2, Silhoue...",114883811619,1994-1995 Nike Air Jordan II 2 Low Retro Chica...,https://www.ebay.com/b/bn_7117647055,https://www.ebay.com/itm/114883811619,y,NaN,NaN,NaN,NaN,NaN
6,7117649456,"[Brand = Jordan, Model = Jordan Flight Remix, ...",184714584239,Air Jordan Flight Remix Wolf Grey/Infrared 23-...,https://www.ebay.com/b/bn_7117649456,https://www.ebay.com/itm/184714584239,y,NaN,NaN,NaN,NaN,NaN
7,7117657277,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",334002458984,Jordan 1 Low TD Shadow 2019 Men’s Size 10 av52...,https://www.ebay.com/b/bn_7117657277,https://www.ebay.com/itm/334002458984,y,NaN,NaN,NaN,NaN,NaN
8,7117648427,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",193887868653,Size 9 - Jordan 1 High Zoom Air Comfort Red,https://www.ebay.com/b/bn_7117648427,https://www.ebay.com/itm/193887868653,y,NaN,NaN,NaN,NaN,NaN
9,7117648509,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",333958246029,Size 8.5 - Jordan 1 Retro High OG Carmine DS S...,https://www.ebay.com/b/bn_7117648509,https://www.ebay.com/itm/333958246029,y,NaN,NaN,NaN,NaN,NaN


In [98]:
# Voy a recorrer todas las filas buscandos los ids
# Para cada id voy a agregar una columna a la tabla mas una columna con el YES/NO total
for ind, row in df.iterrows():
    print("Procesando fila %d"%(ind+1), end='\r')
    
    #Cada 1000 consultas espero un minuto
    if ind > 0 and ind%1000 == 0:
        time.sleep(60)
        
        
    # Obtengo la URL (product_url)
    html = get_html_safe(row['product_url'])
    
    if html is None:
        print("Error obteniendo html. Filas procesadas {}".format(ind-1))
        break
    
    
    # Busco los ids en la url
    for index, (key, value) in enumerate(search_product_url.items()):
        match = find_error_in_html(html, key, value)
        if match is not None:
            #print(match)
            df.loc[ind, key] = "SI" 
            df.loc[ind,"YES/NO Procesado"] = "n"
            break
            
    # Si no encontre ninguno de los errores, posiblemente sea un YES.
    # Busco el nombre del articulo exacto
    if match is None:
        good_link = check_good_link(html, row['product_title (by seller)']) 
        if good_link:
            df.loc[ind,"YES/NO Procesado"] = "y"       
            
        
            
# Guardo los resultado en una nueva planilla
df.to_excel('../results/'+output_filename)
    
df

,Product_line_id,Product_line_description,product_id,product_title (by seller),product_line_url,product_url,YES/NO,YES/NO Procesado,Found Similar,Sold Out,Ended,Error
0,7117656342,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",264737435029,NIKE AIR JORDAN 1 RETRO + NEUTRAL GREY 2001 SI...,https://www.ebay.com/b/bn_7117656342,https://www.ebay.com/itm/264737435029,y,y,NaN,NaN,NaN,NaN
1,7117717160,"[Brand = Jordan, Model = Jordan 'Why Not?' ZER...",164677534195,NEW Nike Jordan Why Not? Zer0.4 FACETASM Unise...,https://www.ebay.com/b/bn_7117717160,https://www.ebay.com/itm/164677534195,y,y,NaN,NaN,NaN,NaN
2,7117656574,"[Brand = Jordan, Model = Air Jordan 13, Silhou...",293808623286,Nike Air Jordan 2010 Retro 13 Varsity White Re...,https://www.ebay.com/b/bn_7117656574,https://www.ebay.com/itm/293808623286,y,y,NaN,NaN,NaN,NaN
3,7117651224,"[Brand = Jordan, Model = Air Jordan 5, Silhoue...",114780501812,2021 Nike Air Jordan Retro 5 Low Chinese New Y...,https://www.ebay.com/b/bn_7117651224,https://www.ebay.com/itm/114780501812,y,y,NaN,NaN,NaN,NaN
4,7117643400,"[Brand = Jordan, Model = Air Jordan 10, Silhou...",334065851300,Size 10 - Jordan 10 Retro NYC 2016,https://www.ebay.com/b/bn_7117643400,https://www.ebay.com/itm/334065851300,y,y,NaN,NaN,NaN,NaN
5,7117647055,"[Brand = Jordan, Model = Air Jordan 2, Silhoue...",114883811619,1994-1995 Nike Air Jordan II 2 Low Retro Chica...,https://www.ebay.com/b/bn_7117647055,https://www.ebay.com/itm/114883811619,y,y,NaN,NaN,NaN,NaN
6,7117649456,"[Brand = Jordan, Model = Jordan Flight Remix, ...",184714584239,Air Jordan Flight Remix Wolf Grey/Infrared 23-...,https://www.ebay.com/b/bn_7117649456,https://www.ebay.com/itm/184714584239,y,y,NaN,NaN,NaN,NaN
7,7117657277,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",334002458984,Jordan 1 Low TD Shadow 2019 Men’s Size 10 av52...,https://www.ebay.com/b/bn_7117657277,https://www.ebay.com/itm/334002458984,y,y,NaN,NaN,NaN,NaN
8,7117648427,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",193887868653,Size 9 - Jordan 1 High Zoom Air Comfort Red,https://www.ebay.com/b/bn_7117648427,https://www.ebay.com/itm/193887868653,y,y,NaN,NaN,NaN,NaN
9,7117648509,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",333958246029,Size 8.5 - Jordan 1 Retro High OG Carmine DS S...,https://www.ebay.com/b/bn_7117648509,https://www.ebay.com/itm/333958246029,y,y,NaN,NaN,NaN,NaN


In [99]:
# Verificamos que filas no pudimos completar automaticamente
procesadas = len(df.index) 
completadas = len(df.loc[~pd.isna(df["YES/NO Procesado"])].index)

print("Total Filas procesadas: {}. Filas completadas: {}".format(procesadas, completadas))

sin_completar = ""
if procesadas > completadas:
    sin_completar = list(df.loc[pd.isna(df["YES/NO Procesado"])].index)

print("Filas sin completar. Revisar manualmente: {}".format(sin_completar))

Total Filas procesadas: 500. Filas completadas: 499
Filas sin completar. Revisar manualmente: [209]


In [100]:
# Comparamos las filas completadas manualmente con las procesadas por el script
# Obtenemos solo las filas que se completaron manualmente
check_df = df.loc[~pd.isna(df['YES/NO'])]

# Primero vemos cuantas entradas no coinciden con lo procesado
comp = np.where((check_df["YES/NO Procesado"] == check_df["YES/NO"]) | (pd.isna(check_df["YES/NO Procesado"])), True, False)

if not all(comp):
    print("Las siguientes filas no coinciden con lo procesado:")
    print(list(np.where(comp == False)[0]))
else:
    print("Todas las filas coinciden!")


Las siguientes filas no coinciden con lo procesado:
[17, 23, 43, 47, 57, 66, 109, 115, 148, 168, 171, 178, 191, 207, 248, 252, 259, 277, 288, 316, 336, 344, 361, 374, 376, 382, 400, 439, 443, 450, 455, 456, 466, 468, 470, 486]


In [101]:
# Listamos las entradas donde la entrada manual es YES y la procesada es NO
comp1 = np.where((check_df["YES/NO"] == "y") & (check_df["YES/NO Procesado"] == "n") , False, True)

if not all(comp1):
    print("Las siguientes filas tienen la entrada manual en YES y la procesada en NO:")
    print(list(np.where(comp1 == False)[0]))
else:
    print("No hay filas con la entrada manual en YES y la procesada en NO")



Las siguientes filas tienen la entrada manual en YES y la procesada en NO:
[17, 47, 57, 115, 148, 168, 171, 178, 191, 207, 248, 252, 259, 277, 288, 316, 336, 344, 361, 376, 439, 443, 450, 455, 456, 468, 470]


In [102]:
# Listamos las entradas donde la entrada manual es NO y la procesada es YES
comp1 = np.where((check_df["YES/NO"] == "n") & (check_df["YES/NO Procesado"] == "y") , False, True)

if not all(comp1):
    print("Las siguientes filas tienen la entrada manual en NO y la procesada en YES:")
    print(list(np.where(comp1 == False)[0]))
else:
    print("No hay filas con la entrada manual en NO y la procesada en YES")

Las siguientes filas tienen la entrada manual en NO y la procesada en YES:
[23, 43, 66, 109, 374, 382, 400, 466, 486]
